In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [2]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake['class']=0
true['class']=1

In [6]:
data = pd.concat([fake, true], axis=0)

In [7]:
data.sample(10)

,title,text,subject,date,class
17190,Xi says China has zero tolerance for corruptio...,BEIJING (Reuters) - China has zero tolerance f...,worldnews,"October 18, 2017",1
16294,EU Commission has no new comment on Catalonia ...,BRUSSELS (Reuters) - The European Commission s...,worldnews,"October 27, 2017",1
20948,Colombia's Golfo crime gang willing to surrend...,BOGOTA (Reuters) - Colombia s notorious Golfo ...,worldnews,"September 5, 2017",1
1836,U.S. House unanimously approves sweeping self-...,WASHINGTON (Reuters) - The U.S. House on Wedne...,politicsNews,"September 6, 2017",1
6385,Texas Teabagger Lt. Governor Promises To Take...,Tea Party radio host and Lieutenant Governor o...,News,"May 14, 2016",0
435,Trump's call for military buildup hits bump in...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"November 30, 2017",1
22487,"SYRIA CEASEFIRE? Lavrov, Kerry Agree to Fight ...",21st Century Wire says After a long drawn out ...,US_News,"September 10, 2016",0
7592,Clinton holds slim lead over Trump in presiden...,WASHINGTON (Reuters) - Democrat Hillary Clinto...,politicsNews,"November 3, 2016",1
3938,FBI's Comey on Trump travel ban: 'citizenship ...,WASHINGTON (Reuters) - FBI Director James Come...,politicsNews,"May 3, 2017",1
16389,Next round of Syria talks in Geneva set for No...,UNITED NATIONS (Reuters) - The next round of t...,worldnews,"October 26, 2017",1


In [9]:
data = data.drop(["title","subject","date"],axis=1)

In [10]:
data.reset_index(inplace=True)

In [11]:
data.drop(['index'],axis=1,inplace=True)

In [12]:
data.sample(5)

,text,class
1112,Trump and his minions and cronies twist realit...,0
39791,NAIROBI (Reuters) - Kenya s President Uhuru Ke...,1
9571,"On June 15, 2016, President Trump warned Ameri...",0
33608,WASHINGTON (Reuters) - U.S. Republican preside...,1
18770,The poll below is why people shouldn t trust p...,0


In [14]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', "", text)
    text = re.sub(r"\\W"," ", text)
    text = re.sub(r"https?:://\S+|www\.\S+", "", text)
    text = re.sub(r"<.*?>+", "", text)
    text = re.sub(r"[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"\w*\d\w*", "", text)
    return text

In [15]:
data["text"] = data["text"].apply(clean_text)

In [16]:
x = data["text"]
y = data["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=42)

In [17]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [18]:
lr = LogisticRegression()
lr.fit(xv_train, ytrain)

LogisticRegression()

In [20]:
prediction = lr.predict(xv_test)
lr.score(xv_test, ytest)

0.9877951002227171

In [21]:
print(classification_report(ytest, prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.99      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [22]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']